In [1]:
import nltk
from nltk.tokenize import word_tokenize
import matplotlib
%matplotlib inline
from pymorphy2 import MorphAnalyzer
from razdel import tokenize, sentenize
import corus
from corus import load_rudrec
from nltk.corpus import brown
from nltk.tag import DefaultTagger
from nltk.tag import UnigramTagger
from nltk.tag import BigramTagger, TrigramTagger
from nltk.tag import RegexpTagger
from nltk.tag import SequentialBackoffTagger
import warnings
warnings.filterwarnings("ignore")

In [2]:
import pyconll
!wget -O ./ru_syntagrus-ud-train.conllu https://github.com/UniversalDependencies/UD_Russian-SynTagRus/raw/master/ru_syntagrus-ud-train-a.conllu
!wget -O ./ru_syntagrus-ud-test.conllu https://github.com/UniversalDependencies/UD_Russian-SynTagRus/raw/master/ru_syntagrus-ud-test.conllu

--2022-10-31 21:52:50--  https://github.com/UniversalDependencies/UD_Russian-SynTagRus/raw/master/ru_syntagrus-ud-train-a.conllu
Распознаётся github.com (github.com)… 140.82.121.3
Подключение к github.com (github.com)|140.82.121.3|:443... соединение установлено.
HTTP-запрос отправлен. Ожидание ответа… 302 Found
Адрес: https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-train-a.conllu [переход]
--2022-10-31 21:52:51--  https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-train-a.conllu
Распознаётся raw.githubusercontent.com (raw.githubusercontent.com)… 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Подключение к raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... соединение установлено.
HTTP-запрос отправлен. Ожидание ответа… 200 OK
Длина: 40736565 (39M) [text/plain]
Сохранение в: «./ru_syntagrus-ud-train.conllu»

./ru_syntagrus-ud-t 100%[===================>]

In [3]:
full_train = pyconll.load_from_file('ru_syntagrus-ud-train.conllu')
full_test = pyconll.load_from_file('ru_syntagrus-ud-test.conllu')

In [4]:
# view data 
for sent in full_train[:2]:
    for token in sent:
        print(token.form, token.upos)
    

Анкета NOUN
. PUNCT
Начальник NOUN
областного ADJ
управления NOUN
связи NOUN
Семен PROPN
Еремеевич PROPN
был AUX
человек NOUN
простой ADJ
, PUNCT
приходил VERB
на ADP
работу NOUN
всегда ADV
вовремя ADV
, PUNCT
здоровался VERB
с ADP
секретаршей NOUN
за ADP
руку NOUN
и CCONJ
иногда ADV
даже PART
писал VERB
в ADP
стенгазету NOUN
заметки NOUN
под ADP
псевдонимом NOUN
" PUNCT
Муха NOUN
" PUNCT
. PUNCT


In [5]:
fdata_train = []
for sent in full_train[:]:
    fdata_train.append([(token.form, token.upos) for token in sent])
    
fdata_test = []
for sent in full_test[:]:
    fdata_test.append([(token.form, token.upos) for token in sent])
    
fdata_sent_test = []
for sent in full_test[:]:
    fdata_sent_test.append([token.form for token in sent])

In [6]:
len(fdata_train), len(fdata_sent_test)

(24516, 8800)

In [7]:
# see what is data
fdata_test[2]

[('С', 'ADP'),
 ('одной', 'NUM'),
 ('стороны', 'NOUN'),
 (',', 'PUNCT'),
 ('квалифицированные', 'VERB'),
 ('кадры', 'NOUN'),
 ('и', 'CCONJ'),
 ('развитая', 'VERB'),
 ('производственная', 'ADJ'),
 ('инфраструктура', 'NOUN'),
 ('резко', 'ADV'),
 ('отличали', 'VERB'),
 ('Армению', 'PROPN'),
 ('от', 'ADP'),
 ('других', 'ADJ'),
 ('регионов', 'NOUN'),
 ('СССР', 'PROPN'),
 (',', 'PUNCT'),
 ('где', 'ADV'),
 ('доминировали', 'VERB'),
 ('добывающие', 'VERB'),
 ('отрасли', 'NOUN'),
 (',', 'PUNCT'),
 ('а', 'CCONJ'),
 ('экономика', 'NOUN'),
 ('строилась', 'VERB'),
 ('на', 'ADP'),
 ('поставке', 'NOUN'),
 ('сырьевых', 'ADJ'),
 ('ресурсов', 'NOUN'),
 ('.', 'PUNCT')]

In [8]:
# view how tags most valued
tags = [sent[0][1] for sent in fdata_train]
nltk.FreqDist(tags).most_common()

[('ADP', 3831),
 ('NOUN', 3396),
 ('ADV', 3356),
 ('CCONJ', 2652),
 ('ADJ', 2130),
 ('PRON', 2108),
 ('VERB', 1781),
 ('PUNCT', 1677),
 ('PART', 862),
 ('DET', 846),
 ('SCONJ', 820),
 ('PROPN', 650),
 ('NUM', 330),
 ('AUX', 47),
 ('INTJ', 21),
 ('X', 5),
 (None, 4)]

In [9]:
# test DefaultTagger
default_tagger = nltk.DefaultTagger('NOUN')
display(default_tagger.tag(fdata_sent_test[2]), default_tagger.evaluate(fdata_test))

[('С', 'NOUN'),
 ('одной', 'NOUN'),
 ('стороны', 'NOUN'),
 (',', 'NOUN'),
 ('квалифицированные', 'NOUN'),
 ('кадры', 'NOUN'),
 ('и', 'NOUN'),
 ('развитая', 'NOUN'),
 ('производственная', 'NOUN'),
 ('инфраструктура', 'NOUN'),
 ('резко', 'NOUN'),
 ('отличали', 'NOUN'),
 ('Армению', 'NOUN'),
 ('от', 'NOUN'),
 ('других', 'NOUN'),
 ('регионов', 'NOUN'),
 ('СССР', 'NOUN'),
 (',', 'NOUN'),
 ('где', 'NOUN'),
 ('доминировали', 'NOUN'),
 ('добывающие', 'NOUN'),
 ('отрасли', 'NOUN'),
 (',', 'NOUN'),
 ('а', 'NOUN'),
 ('экономика', 'NOUN'),
 ('строилась', 'NOUN'),
 ('на', 'NOUN'),
 ('поставке', 'NOUN'),
 ('сырьевых', 'NOUN'),
 ('ресурсов', 'NOUN'),
 ('.', 'NOUN')]

0.23145915221945831

In [10]:
# now look at UnigramTagger
unigram_tagger = UnigramTagger(fdata_train)
display(unigram_tagger.tag(fdata_sent_test[2]), unigram_tagger.evaluate(fdata_test))

[('С', 'ADP'),
 ('одной', 'NUM'),
 ('стороны', 'NOUN'),
 (',', 'PUNCT'),
 ('квалифицированные', 'ADJ'),
 ('кадры', 'NOUN'),
 ('и', 'CCONJ'),
 ('развитая', 'VERB'),
 ('производственная', None),
 ('инфраструктура', 'NOUN'),
 ('резко', 'ADV'),
 ('отличали', 'VERB'),
 ('Армению', None),
 ('от', 'ADP'),
 ('других', 'ADJ'),
 ('регионов', 'NOUN'),
 ('СССР', 'PROPN'),
 (',', 'PUNCT'),
 ('где', 'ADV'),
 ('доминировали', None),
 ('добывающие', None),
 ('отрасли', 'NOUN'),
 (',', 'PUNCT'),
 ('а', 'CCONJ'),
 ('экономика', 'NOUN'),
 ('строилась', 'VERB'),
 ('на', 'ADP'),
 ('поставке', None),
 ('сырьевых', 'ADJ'),
 ('ресурсов', 'NOUN'),
 ('.', 'PUNCT')]

0.8369316851173183

In [11]:
# made bigram_tagger on basis unigram_tagger
bigram_tagger = BigramTagger(fdata_train, backoff=unigram_tagger)
display(bigram_tagger.tag(fdata_sent_test[2]), bigram_tagger.evaluate(fdata_test))

[('С', 'ADP'),
 ('одной', 'NUM'),
 ('стороны', 'NOUN'),
 (',', 'PUNCT'),
 ('квалифицированные', 'ADJ'),
 ('кадры', 'NOUN'),
 ('и', 'CCONJ'),
 ('развитая', 'VERB'),
 ('производственная', None),
 ('инфраструктура', 'NOUN'),
 ('резко', 'ADV'),
 ('отличали', 'VERB'),
 ('Армению', None),
 ('от', 'ADP'),
 ('других', 'ADJ'),
 ('регионов', 'NOUN'),
 ('СССР', 'PROPN'),
 (',', 'PUNCT'),
 ('где', 'ADV'),
 ('доминировали', None),
 ('добывающие', None),
 ('отрасли', 'NOUN'),
 (',', 'PUNCT'),
 ('а', 'CCONJ'),
 ('экономика', 'NOUN'),
 ('строилась', 'VERB'),
 ('на', 'ADP'),
 ('поставке', None),
 ('сырьевых', 'ADJ'),
 ('ресурсов', 'NOUN'),
 ('.', 'PUNCT')]

0.8419889992341238

In [12]:
# build trigram_tagger on basis unigramm_tagger
trigram_tagger_unigram = TrigramTagger(fdata_train, backoff=unigram_tagger)
display(trigram_tagger_unigram.tag(fdata_sent_test[2]), trigram_tagger_unigram.evaluate(fdata_train))

[('С', 'ADP'),
 ('одной', 'NUM'),
 ('стороны', 'NOUN'),
 (',', 'PUNCT'),
 ('квалифицированные', 'ADJ'),
 ('кадры', 'NOUN'),
 ('и', 'CCONJ'),
 ('развитая', 'VERB'),
 ('производственная', None),
 ('инфраструктура', 'NOUN'),
 ('резко', 'ADV'),
 ('отличали', 'VERB'),
 ('Армению', None),
 ('от', 'ADP'),
 ('других', 'ADJ'),
 ('регионов', 'NOUN'),
 ('СССР', 'PROPN'),
 (',', 'PUNCT'),
 ('где', 'ADV'),
 ('доминировали', None),
 ('добывающие', None),
 ('отрасли', 'NOUN'),
 (',', 'PUNCT'),
 ('а', 'CCONJ'),
 ('экономика', 'NOUN'),
 ('строилась', 'VERB'),
 ('на', 'ADP'),
 ('поставке', None),
 ('сырьевых', 'ADJ'),
 ('ресурсов', 'NOUN'),
 ('.', 'PUNCT')]

0.9848093068219681

In [13]:
# build trigram_tagger on basis bigramm_tagger
trigram_tagger_unigram = TrigramTagger(fdata_train, backoff=bigram_tagger)
display(trigram_tagger_unigram.tag(fdata_sent_test[2]), trigram_tagger_unigram.evaluate(fdata_train))

[('С', 'ADP'),
 ('одной', 'NUM'),
 ('стороны', 'NOUN'),
 (',', 'PUNCT'),
 ('квалифицированные', 'ADJ'),
 ('кадры', 'NOUN'),
 ('и', 'CCONJ'),
 ('развитая', 'VERB'),
 ('производственная', None),
 ('инфраструктура', 'NOUN'),
 ('резко', 'ADV'),
 ('отличали', 'VERB'),
 ('Армению', None),
 ('от', 'ADP'),
 ('других', 'ADJ'),
 ('регионов', 'NOUN'),
 ('СССР', 'PROPN'),
 (',', 'PUNCT'),
 ('где', 'ADV'),
 ('доминировали', None),
 ('добывающие', None),
 ('отрасли', 'NOUN'),
 (',', 'PUNCT'),
 ('а', 'CCONJ'),
 ('экономика', 'NOUN'),
 ('строилась', 'VERB'),
 ('на', 'ADP'),
 ('поставке', None),
 ('сырьевых', 'ADJ'),
 ('ресурсов', 'NOUN'),
 ('.', 'PUNCT')]

0.984957130990985

##### Попробуем комбинацию разных taggers

In [14]:
def combine_tagger(train_data, tagger_class, backoff=None):
    for cls in tagger_class:
        backoff = cls(train_data, backoff = backoff)
    
    return backoff

backoff = DefaultTagger('NOUN')
tagger = combine_tagger(fdata_train, [UnigramTagger, BigramTagger, TrigramTagger], backoff = backoff)
tagger.evaluate(fdata_test)

0.8893720448892012

#### Попробуем создать свой собственный тэггер

In [15]:
# create corpus with PROPN word for own tagger
propn = []
morph = MorphAnalyzer()

for sent in fdata_train:
    #print([word[0] for word in sent if word[1] == 'PROPN'])
    for word in sent:
        if word[1] == 'PROPN':
            new_propn = morph.normal_forms(word[0].lower())[0]
            if new_propn not in propn:
                propn.append(new_propn)
propn[:10]

['семён',
 'еремей',
 'семя',
 'ефимов',
 'москва',
 'юлий',
 'цезарь',
 'рим',
 'евгений',
 'василий']

In [16]:
class CustomTagger(SequentialBackoffTagger):
    def __init__(self, *arg, **kwargs):
        SequentialBackoffTagger.__init__(self, *arg, **kwargs)
        self.tags_set = set([n for n in propn])
        
    def choose_tag(self, tokens, index, history):
        word = tokens[index]
        if word.lower() in self.tags_set:
            return 'TARGET'
        else:
            return None
        
        

In [17]:
my_tagger = CustomTagger()

print(my_tagger.tag(['Юлий']))
print(my_tagger.tag(['Россия']))
print(my_tagger.tag(['Пермь']))
print(my_tagger.tag(['Тайланд']))
print(my_tagger.tag(['Кипр']))

[('Юлий', 'TARGET')]
[('Россия', 'TARGET')]
[('Пермь', 'TARGET')]
[('Тайланд', None)]
[('Кипр', 'TARGET')]


### Вывод по работе с POS-разметкой текста:
Решение проблемы с поиском части речи в тексте является одной из главных на пути понимания и интерпретации больших объемов текстовой информации. С помощью POS разметки текста мы сможем глубже понимать контектс информации.
Одним из механизмов POS разметки - nltk.tag. В этой библиотеке существует несколько классов taggers: DefaultTagger, UnigramTagger, BigramTagger, TrigramTagger. Самый просто (и самый бестолковый) это DefaultTagger он просто присваиват всем словам указаный tag. Далее результат предсказания повышается по мере увеличение слов задействованных в окне контекста. По этому логичным получился результат, что лучшим оказался TrigrammTagger показавший самый высокий результат в предсказании. Однако все модели ошиблись на словах 'доминировали','добывающие'. Ни один tagger не смог определить их принадлежность. Что странно.

Этот механизм - поиск нужных слов в больших объемах текстовых данных очень удобен тем, что можно разработать свой собственный теггер с каким угодно правилом отбора.

### Работа с NER
<b>NER</b> - это выделение в тексте именованных сущностей таких как названия организаций, имена знаметиный людей, названия городов, стран и т.д. 
В общем это очень важная часть работы с текстовой информацией.
Попробуем как с этой задачей справляются библиотки nltk,natasha.

In [18]:
#test_article = [line for line in file()]
test_article = ""

with open('./Загрузки/Collection3/last_70.txt') as file:
    test_article = [line for line in file]
test_article = '\n'.join(test_article)
test_article = test_article.replace('\n','')
test_article = test_article.replace('ЧукоткиПрезидент', 'Чукотки. Президент')
test_article

'В.Путин оставил Р.Копина во главе Чукотки. Президент России Владимир Путин своим указом назначил временно исполняющим обязанности губернатора Чукотского автономного округа Романа Копина. Указ подписан в связи с истечением срока его полномочий на посту губернатора, сообщила пресс-служба Кремля.Р.Копин назначен временным главой региона до вступления в должность лица, избранного губернатором Чукотки. Президентский указ вступает в силу со дня его подписания.Р.Копин официально вступил в должность губернатора Чукотки в июле 2008г. после отставки Романа Абрамовича. Его кандидатуру в качестве главы региона предложил тогда еще президент России Дмитрий Медведев.До назначения Р.Копин занимал посты заместителя губернатора и заместителя председателя правительства округа по вопросам промышленной и сельскохозяйственной политики.'

In [19]:
nltk.download('maxent_ne_chunker')
nltk.download('words')

[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /home/medic/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /home/medic/nltk_data...
[nltk_data]   Package words is already up-to-date!


True

#### nltk

In [20]:
{(' '.join(c[0] for c in chunk), chunk.label() ) for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(test_article))) if hasattr(chunk, 'label') }

{('Романа Абрамовича', 'PERSON'),
 ('Романа Копина', 'PERSON'),
 ('России Владимир Путин', 'PERSON'),
 ('России Дмитрий', 'PERSON'),
 ('Чукотки', 'PERSON')}

#### natasha

In [21]:
from navec import Navec
from slovnet import NER, Syntax
from ipymarkup import show_span_ascii_markup as show_markup
from razdel import sentenize, tokenize

navec = Navec.load('navec_news_v1_1B_250K_300d_100q.tar')
ner = NER.load('slovnet_ner_news_v1.tar')
ner.navec(navec)
markup = ner(test_article)
show_markup(markup.text, markup.spans)

В.Путин оставил Р.Копина во главе Чукотки. Президент России Владимир 
PER────         PER─────          LOC────            LOC─── PER──────
Путин своим указом назначил временно исполняющим обязанности 
─────                                                        
губернатора Чукотского автономного округа Романа Копина. Указ подписан
            LOC────────────────────────── PER──────────               
 в связи с истечением срока его полномочий на посту губернатора, 
сообщила пресс-служба Кремля.Р.Копин назначен временным главой региона
                      LOC─── PER────                                  
 до вступления в должность лица, избранного губернатором Чукотки. 
                                                         LOC────  
Президентский указ вступает в силу со дня его подписания.Р.Копин 
                                                         PER──── 
официально вступил в должность губернатора Чукотки в июле 2008г. после
                                           LOC───

Попробуем извлечение отношений <b>NER</b>

#### natasha

In [22]:
from natasha import (
    Segmenter,
    MorphVocab,
    
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    
    PER,
    NamesExtractor,

    Doc
)

segmenter = Segmenter()
morph_vocab = MorphVocab()

emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)

names_extractor = NamesExtractor(morph_vocab)

doc = Doc(test_article)
doc.segment(segmenter)
doc.tag_morph(morph_tagger)
doc.parse_syntax(syntax_parser)

In [23]:
doc.sents[3].syntax.print()

              ┌► Президентский amod
            ┌►└─ указ          nsubj
┌─┌───────┌─└─── вступает      
│ │       │   ┌► в             case
│ │       └──►└─ силу          obl
│ │           ┌► со            case
│ │     ┌──►┌─└─ дня           obl
│ │     │   │ ┌► его           nmod
│ │     │   └►└─ подписания    nmod
│ │     │     ┌► .             punct
│ │     │ ┌─┌─└─ Р             
│ │     │ │ └──► .             punct
│ │     │ └────► Копин         flat:name
│ │     │     ┌► официально    advmod
│ │ ┌─┌─└─┌───└─ вступил       
│ │ │ │   │   ┌► в             case
│ └►│ │   │ ┌─└─ должность     obl
│   │ │   │ └►┌─ губернатора   nmod
│   │ │   │   └► Чукотки       nmod
│   │ │   │   ┌► в             case
│   │ │   └►┌─└─ июле          obl
│   │ │     │ ┌► 2008          amod
│   │ │     └►└─ г             nmod
│   │ └────────► .             punct
│   │         ┌► после         case
│   └──────►┌─└─ отставки      obl
│           └►┌─ Романа        nmod
│             └► Абрамовича    fla

##### Попробуем построить свою модель распознавания NER

In [24]:
!wget https://github.com/cimm-kzn/RuDReC/raw/master/data/rudrec_annotated.json

--2022-10-31 21:53:46--  https://github.com/cimm-kzn/RuDReC/raw/master/data/rudrec_annotated.json
Распознаётся github.com (github.com)… 140.82.121.3
Подключение к github.com (github.com)|140.82.121.3|:443... соединение установлено.
HTTP-запрос отправлен. Ожидание ответа… 302 Found
Адрес: https://raw.githubusercontent.com/cimm-kzn/RuDReC/master/data/rudrec_annotated.json [переход]
--2022-10-31 21:53:47--  https://raw.githubusercontent.com/cimm-kzn/RuDReC/master/data/rudrec_annotated.json
Распознаётся raw.githubusercontent.com (raw.githubusercontent.com)… 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Подключение к raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... соединение установлено.
HTTP-запрос отправлен. Ожидание ответа… 200 OK
Длина: 1773014 (1,7M) [text/plain]
Сохранение в: «rudrec_annotated.json.2»

rudrec_annotated.js 100%[===================>]   1,69M  5,72MB/s    за 0,3s    

2022-10-31 21:53:47 (5,72 MB/s) - «rudrec_annotated.json.2» сохра

In [25]:
path = 'rudrec_annotated.json'
records = load_rudrec(path)
next(records)

RuDReCRecord(
    file_name='172744.tsv',
    text='нам прописали, так мой ребенок сыпью покрылся, глаза опухли, сверху и снизу на веках высыпала сыпь, ( 8 месяцев сыну)А от виферона такого не было... У кого ещё такие побочки, отзовитесь!1 Чем спасались?\n',
    sentence_id=0,
    entities=[RuDReCEntity(
         entity_id='*[0]_se',
         entity_text='виферона',
         entity_type='Drugform',
         start=122,
         end=130,
         concept_id='C0021735',
         concept_name=nan
     ),
     RuDReCEntity(
         entity_id='*[1]',
         entity_text='сыпью покрылся',
         entity_type='ADR',
         start=31,
         end=45,
         concept_id='C0015230',
         concept_name=nan
     ),
     RuDReCEntity(
         entity_id='*[2]',
         entity_text='глаза опухли',
         entity_type='ADR',
         start=47,
         end=59,
         concept_id='C4760994',
         concept_name=nan
     ),
     RuDReCEntity(
         entity_id='*[3]',
         entity_text

In [26]:
docs = []
for rec in load_rudrec(path):
    words = []
    labels = []
    idx_ent = -1
    len_ents = len(rec.entities)
    rec_entities = sorted(rec.entities, key=lambda v: v.start)
    ent = None
    is_start = None
    for token in tokenize(rec.text):
        type_ent = 'OUT'
        if len_ents == 0:
            words.append(token.text)
            labels.append(type_ent)
            continue

        if (idx_ent == -1) or (idx_ent + 1 < len_ents and token.start > ent.end):
            idx_ent += 1
            ent = rec_entities[idx_ent]
            is_start = True

        if (token.start >= ent.start) and (token.stop <= ent.end):
                type_ent = 'B-' + ent.entity_type if is_start else 'I-' + ent.entity_type
                is_start = False
        words.append(token.text)
        labels.append(type_ent)
    
    docs.append([words, labels])

In [27]:
print(docs[0][0])
print(docs[0][1])

['нам', 'прописали', ',', 'так', 'мой', 'ребенок', 'сыпью', 'покрылся', ',', 'глаза', 'опухли', ',', 'сверху', 'и', 'снизу', 'на', 'веках', 'высыпала', 'сыпь', ',', '(', '8', 'месяцев', 'сыну', ')', 'А', 'от', 'виферона', 'такого', 'не', 'было', '...', 'У', 'кого', 'ещё', 'такие', 'побочки', ',', 'отзовитесь', '!', '1', 'Чем', 'спасались', '?']
['OUT', 'OUT', 'OUT', 'OUT', 'OUT', 'OUT', 'B-ADR', 'I-ADR', 'OUT', 'B-ADR', 'I-ADR', 'OUT', 'OUT', 'OUT', 'OUT', 'B-ADR', 'I-ADR', 'I-ADR', 'I-ADR', 'OUT', 'OUT', 'OUT', 'OUT', 'OUT', 'OUT', 'OUT', 'OUT', 'B-Drugform', 'OUT', 'OUT', 'OUT', 'OUT', 'OUT', 'OUT', 'OUT', 'OUT', 'OUT', 'OUT', 'OUT', 'OUT', 'OUT', 'OUT', 'OUT', 'OUT']


In [28]:
from IPython.display import Image
from IPython.core.display import display, HTML
import pandas as pd
from collections import Counter
import random
from spacy import displacy
import json
import os
from spacy.training import offsets_to_biluo_tags
import spacy
import eli5
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import sklearn
import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics
from sklearn.metrics import f1_score

In [29]:
len(docs)

4809

In [30]:
with open('rudrec.bio', 'w') as w:
    with open('rudrec_train.bio', 'w') as w1:
        with open('rudrec_valid.bio', 'w') as w2:
            for irec, rec in enumerate(docs):
                for line in map(lambda vl: '\t'.join(vl) + '\n', zip(*rec)):
                    w.write(line)
                    if irec < 3000:
                        w1.write(line)
                    else:
                        w2.write(line)
                w.write('\n')
                if irec < 3000:
                    w1.write('\n')
                else:
                    w2.write('\n')

In [31]:
data, labels = list(zip(*docs))

In [32]:
for w, e in zip(data[0], labels[0]):
    print(f'{w}\t{e}')

нам	OUT
прописали	OUT
,	OUT
так	OUT
мой	OUT
ребенок	OUT
сыпью	B-ADR
покрылся	I-ADR
,	OUT
глаза	B-ADR
опухли	I-ADR
,	OUT
сверху	OUT
и	OUT
снизу	OUT
на	B-ADR
веках	I-ADR
высыпала	I-ADR
сыпь	I-ADR
,	OUT
(	OUT
8	OUT
месяцев	OUT
сыну	OUT
)	OUT
А	OUT
от	OUT
виферона	B-Drugform
такого	OUT
не	OUT
было	OUT
...	OUT
У	OUT
кого	OUT
ещё	OUT
такие	OUT
побочки	OUT
,	OUT
отзовитесь	OUT
!	OUT
1	OUT
Чем	OUT
спасались	OUT
?	OUT


In [33]:
df = pd.DataFrame({'sent_id': [i for j in [[i] * len(s) for i, s in enumerate(data)] for i in j],
                   'data': [i for j in data for i in j],
                   'entities': [i for j in labels for i in j]})
df.head(50)

,sent_id,data,entities
0,0,нам,OUT
1,0,прописали,OUT
2,0,",",OUT
3,0,так,OUT
4,0,мой,OUT
5,0,ребенок,OUT
6,0,сыпью,B-ADR
7,0,покрылся,I-ADR
8,0,",",OUT
9,0,глаза,B-ADR


In [34]:
class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, t) for w, t in zip(s['data'].values.tolist(), 
                                                           s['entities'].values.tolist())]
        self.grouped = self.data.groupby('sent_id').apply(agg_func)
        self.sentences = [s for s in self.grouped]
        
    def get_next(self):
        try: 
            s = self.grouped['Sentence: {}'.format(self.n_sent)]
            self.n_sent += 1
            return s 
        except:
            return None

getter = SentenceGetter(df)
sentences = getter.sentences

In [35]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]
    
    features = {
        'bias': 1.0, 
        'word.lower()': word.lower(), 
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isdigit()': word.isdigit()
    }
    if i > 0:
        word1 = sent[i - 1][0]
        features.update({
            '-1:word.lower()': word1.lower()
        })
    else:
        features['BOS'] = True
    if i < len(sent) - 1:
        word1 = sent[i + 1][0]
        features.update({
            '+1:word.lower()': word1.lower()
        })
    else:
        features['EOS'] = True

    return features

def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, label in sent]

def sent2tokens(sent):
    return [token for token, label in sent]

In [36]:
X = [sent2features(s) for s in sentences]
y = [sent2labels(s) for s in sentences]

In [37]:
X[4][5]

{'bias': 1.0,
 'word.lower()': 'причем',
 'word[-3:]': 'чем',
 'word[-2:]': 'ем',
 'word.isdigit()': False,
 '-1:word.lower()': 'раз',
 '+1:word.lower()': 'со'}

In [38]:
X_train = X[:3000]
X_test = X[3000:]
y_train = y[:3000]
y_test = y[3000:]

In [39]:
%%time
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=1000,
    all_possible_transitions=True,
    verbose=True
)
crf.fit(X_train, y_train)

loading training data to CRFsuite: 100%|██████████| 3000/3000 [00:00<00:00, 11756.84it/s]



Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 1
0....1....2....3....4....5....6....7....8....9....10
Number of features: 30517
Seconds required: 0.063

L-BFGS optimization
c1: 0.100000
c2: 0.100000
num_memories: 6
max_iterations: 1000
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

Iter 1   time=0.06  loss=57505.29 active=30343 feature_norm=1.00
Iter 2   time=0.07  loss=38940.59 active=30270 feature_norm=5.18
Iter 3   time=0.04  loss=33752.98 active=28909 feature_norm=4.55
Iter 4   time=0.19  loss=20255.77 active=29148 feature_norm=2.82
Iter 5   time=0.03  loss=19925.96 active=29069 feature_norm=3.37
Iter 6   time=0.03  loss=18573.95 active=27358 feature_norm=4.00
Iter 7   time=0.03  loss=17509.89 active=25900 feature_norm=5.45
Iter 8   time=0.03  loss=15172.50 active=22442 feature_norm=9.92
Iter 9   time=0.03  loss=13984.62 active=26449 feature_norm=11.44
Iter 10  tim

Iter 124 time=0.06  loss=3149.84  active=11775 feature_norm=101.87
Iter 125 time=0.03  loss=3149.78  active=11775 feature_norm=101.87
Iter 126 time=0.04  loss=3149.75  active=11773 feature_norm=101.88
Iter 127 time=0.04  loss=3149.56  active=11776 feature_norm=101.87
Iter 128 time=0.06  loss=3149.48  active=11779 feature_norm=101.87
Iter 129 time=0.03  loss=3149.44  active=11777 feature_norm=101.87
Iter 130 time=0.03  loss=3149.39  active=11779 feature_norm=101.88
Iter 131 time=0.03  loss=3149.21  active=11778 feature_norm=101.87
Iter 132 time=0.03  loss=3149.14  active=11779 feature_norm=101.88
Iter 133 time=0.03  loss=3149.01  active=11780 feature_norm=101.87
Iter 134 time=0.03  loss=3149.00  active=11778 feature_norm=101.88
Iter 135 time=0.03  loss=3148.85  active=11782 feature_norm=101.87
Iter 136 time=0.03  loss=3148.84  active=11779 feature_norm=101.88
Iter 137 time=0.03  loss=3148.69  active=11784 feature_norm=101.87
Iter 138 time=0.03  loss=3148.69  active=11780 feature_norm=10

Iter 247 time=0.06  loss=3144.72  active=11678 feature_norm=101.79
Iter 248 time=0.06  loss=3144.71  active=11677 feature_norm=101.80
Iter 249 time=0.06  loss=3144.69  active=11677 feature_norm=101.80
Iter 250 time=0.07  loss=3144.69  active=11678 feature_norm=101.80
Iter 251 time=0.06  loss=3144.66  active=11675 feature_norm=101.80
Iter 252 time=0.06  loss=3144.66  active=11675 feature_norm=101.80
Iter 253 time=0.06  loss=3144.64  active=11674 feature_norm=101.79
Iter 254 time=0.03  loss=3144.62  active=11670 feature_norm=101.80
Iter 255 time=0.06  loss=3144.59  active=11667 feature_norm=101.80
Iter 256 time=0.06  loss=3144.58  active=11667 feature_norm=101.80
Iter 257 time=0.06  loss=3144.56  active=11668 feature_norm=101.80
Iter 258 time=0.06  loss=3144.55  active=11663 feature_norm=101.80
Iter 259 time=0.07  loss=3144.53  active=11662 feature_norm=101.80
Iter 260 time=0.07  loss=3144.52  active=11662 feature_norm=101.80
Iter 261 time=0.03  loss=3144.52  active=11662 feature_norm=10

Iter 370 time=0.06  loss=3143.47  active=11621 feature_norm=101.81
Iter 371 time=0.06  loss=3143.47  active=11621 feature_norm=101.81
Iter 372 time=0.06  loss=3143.46  active=11621 feature_norm=101.81
Iter 373 time=0.06  loss=3143.46  active=11623 feature_norm=101.81
Iter 374 time=0.06  loss=3143.45  active=11623 feature_norm=101.81
Iter 375 time=0.06  loss=3143.45  active=11623 feature_norm=101.81
Iter 376 time=0.06  loss=3143.44  active=11621 feature_norm=101.81
Iter 377 time=0.06  loss=3143.44  active=11621 feature_norm=101.81
Iter 378 time=0.06  loss=3143.44  active=11621 feature_norm=101.81
Iter 379 time=0.06  loss=3143.43  active=11620 feature_norm=101.81
Iter 380 time=0.06  loss=3143.43  active=11620 feature_norm=101.81
Iter 381 time=0.06  loss=3143.43  active=11620 feature_norm=101.81
Iter 382 time=0.06  loss=3143.42  active=11619 feature_norm=101.81
Iter 383 time=0.07  loss=3143.42  active=11619 feature_norm=101.81
Iter 384 time=0.06  loss=3143.41  active=11619 feature_norm=10

CRF(algorithm='lbfgs', all_possible_transitions=True, c1=0.1, c2=0.1,
    keep_tempfiles=None, max_iterations=1000, verbose=True)

In [40]:
all_entities = sorted(df.entities.unique().tolist())

In [41]:
y_pred = crf.predict(X_test)
metrics.flat_f1_score(y_test, y_pred, average='weighted', labels=all_entities)

0.9230928362797047

In [42]:
print(metrics.flat_classification_report(y_test, y_pred, labels = all_entities))

              precision    recall  f1-score   support

       B-ADR       0.57      0.25      0.35       319
        B-DI       0.67      0.46      0.54       509
 B-Drugclass       0.98      0.48      0.64       126
  B-Drugform       0.96      0.78      0.86       303
  B-Drugname       0.90      0.68      0.78       376
   B-Finding       0.33      0.05      0.08       104
       I-ADR       0.51      0.15      0.24       357
        I-DI       0.45      0.21      0.29       345
 I-Drugclass       0.00      0.00      0.00         1
  I-Drugform       0.00      0.00      0.00         1
  I-Drugname       0.95      0.51      0.67        37
   I-Finding       0.00      0.00      0.00        50
         OUT       0.95      0.99      0.97     23102

    accuracy                           0.94     25630
   macro avg       0.56      0.35      0.42     25630
weighted avg       0.92      0.94      0.92     25630



In [43]:
eli5.show_weights(crf, top=10)

From \ To,OUT,B-ADR,I-ADR,B-DI,I-DI,B-Drugclass,B-Drugform,I-Drugform,B-Drugname,I-Drugname,B-Finding,I-Finding
OUT,2.694,1.747,-3.342,2.409,-3.451,1.811,2.001,-1.576,1.426,-2.437,1.881,-2.74
B-ADR,0.037,-0.169,5.912,-1.122,-0.721,-0.429,-0.685,0.0,-1.099,-0.202,-0.244,-0.186
I-ADR,-0.246,0.0,6.157,-0.547,-0.484,-0.04,-0.314,0.0,-0.717,0.0,0.0,0.0
B-DI,0.825,-1.228,-0.689,-0.154,5.804,0.0,-0.973,0.0,-0.481,-0.245,-0.597,-0.536
I-DI,0.102,0.416,-0.673,-1.326,5.562,-0.341,-1.062,0.0,-0.416,-0.132,-0.46,-0.237
B-Drugclass,1.012,-0.57,-0.345,-0.69,-0.289,0.0,1.832,0.0,0.459,-0.04,-0.067,-0.187
B-Drugform,0.905,-0.476,-0.633,-0.247,-0.607,0.0,-0.438,1.914,2.046,-0.476,-0.258,-0.41
I-Drugform,-1.217,0.0,0.0,0.0,0.0,0.0,0.0,1.634,0.657,0.0,0.0,0.0
B-Drugname,1.059,0.0,-0.426,-0.91,-0.426,-0.686,0.26,0.0,0.0,4.321,-0.226,-0.233
I-Drugname,-0.188,-0.042,0.0,-0.278,0.0,-0.302,-0.311,0.0,-0.473,4.679,0.813,0.0


In [44]:
!wget http://www.labinform.ru/pub/named_entities/collection5.zip

--2022-10-31 21:54:14--  http://www.labinform.ru/pub/named_entities/collection5.zip
Распознаётся www.labinform.ru (www.labinform.ru)… 95.181.230.181
Подключение к www.labinform.ru (www.labinform.ru)|95.181.230.181|:80... соединение установлено.
HTTP-запрос отправлен. Ожидание ответа… 200 OK
Длина: 1899530 (1,8M) [application/zip]
Сохранение в: «collection5.zip.2»

collection5.zip.2   100%[===================>]   1,81M  7,19MB/s    за 0,3s    

2022-10-31 21:54:14 (7,19 MB/s) - «collection5.zip.2» сохранён [1899530/1899530]



In [45]:
!unzip collection5.zip

Archive:  collection5.zip
replace Collection5/001.ann? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


In [46]:
from corus import load_ne5

dir = 'Collection5/'
records = load_ne5(dir)
#next(records)

In [47]:
words_docs = []
for ix, rec in enumerate(records):
    words = []
    for token in tokenize(rec.text):
        type_ent = 'OUT'
        for ent in rec.spans:
            if (token.start >= ent.start) and (token.stop <= ent.stop):
                type_ent = ent.type
                break
        words.append([token.text, type_ent])
    words_docs.extend(words)

In [48]:
df_words = pd.DataFrame(words_docs, columns=['word', 'tag'])
df_words['tag'].value_counts()

OUT         219214
PER          21200
ORG          13651
LOC           4568
GEOPOLIT      4356
MEDIA         2482
Name: tag, dtype: int64

In [49]:
df_words.head(7)

,word,tag
0,Чиновник,OUT
1,Ленобласти,LOC
2,:,OUT
3,Нас,OUT
4,беспокоит,OUT
5,",",OUT
6,когда,OUT


In [50]:
df_words.shape

(265471, 2)

In [51]:
import tensorflow as tf

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D, GlobalMaxPooling1D, Conv1D, GRU, LSTM, Dropout, Input
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

In [52]:
from sklearn import model_selection, preprocessing, linear_model

train_x, valid_x, train_y, valid_y = model_selection.train_test_split(df_words['word'], df_words['tag'])

# labelEncode целевую переменную
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)

In [53]:
train_data = tf.data.Dataset.from_tensor_slices((train_x, train_y))
valid_data = tf.data.Dataset.from_tensor_slices((valid_x, valid_y))

train_data = train_data.batch(16)
valid_data = valid_data.batch(16)

In [54]:
AUTOTUNE = tf.data.AUTOTUNE

train_data = train_data.cache().prefetch(buffer_size=AUTOTUNE)
valid_data = valid_data.cache().prefetch(buffer_size=AUTOTUNE)

In [55]:
def custom_standardization(input_data):
    return input_data

vocab_size = 30000
seq_len = 10

vectorize_layer = TextVectorization(
    standardize=custom_standardization,
    max_tokens=vocab_size,
    output_mode='int',
    #ngrams=(1, 3),
    output_sequence_length=seq_len)

# Make a text-only dataset (no labels) and call adapt to build the vocabulary.
text_data = train_data.map(lambda x, y: x)
vectorize_layer.adapt(text_data)

In [56]:
embedding_dim = 64

class modelNER(tf.keras.Model):
    def __init__(self):
        super(modelNER, self).__init__()
        self.emb = Embedding(vocab_size, embedding_dim)
        self.gPool = GlobalMaxPooling1D()
        self.fc1 = Dense(300, activation='relu')
        self.fc2 = Dense(50, activation='relu')
        self.fc3 = Dense(6, activation='softmax')

    def call(self, x):
        x = vectorize_layer(x)
        x = self.emb(x)
        pool_x = self.gPool(x)
        
        fc_x = self.fc1(pool_x)
        fc_x = self.fc2(fc_x)
        
        concat_x = tf.concat([pool_x, fc_x], axis=1)
        prob = self.fc3(concat_x)
        return prob

In [57]:
mmodel = modelNER()
mmodel.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

In [58]:
mmodel.fit(train_data, validation_data=valid_data, epochs=3)

Epoch 1/3
12444/12444 [==============================] - 393s 31ms/step - loss: 0.2960 - accuracy: 0.9140 - val_loss: 0.2076 - val_accuracy: 0.9389
Epoch 2/3
12444/12444 [==============================] - 443s 36ms/step - loss: 0.1283 - accuracy: 0.9621 - val_loss: 0.2124 - val_accuracy: 0.9418
Epoch 3/3
12444/12444 [==============================] - 321s 26ms/step - loss: 0.1109 - accuracy: 0.9651 - val_loss: 0.2707 - val_accuracy: 0.8954


In [59]:
y_pred_one = mmodel.predict(valid_x)
y_pred_one = np.argmax(y_pred_one, -1)

In [60]:
# metrics for prediction
precision = sklearn.metrics.precision_score(valid_y, y_pred_one, average = 'weighted')
recall = sklearn.metrics.recall_score(valid_y, y_pred_one, average = 'weighted')
f1_score = sklearn.metrics.f1_score(valid_y, y_pred_one, average = 'weighted')
precision, recall, f1_score

(0.9213002456095268, 0.895386330761813, 0.9016465477253166)

##### Теперь попробуем обучить модель не по одному токену а с несколькими соседями

In [61]:
dir = 'Collection5/'
records = load_ne5(dir)

In [62]:
words_for_model = []

for ix, rec in enumerate(records):
    words = []
    for ent in rec.spans:          
        tokens = list(rec.text.split())
        if ent.text in tokens:
            if (len(tokens) > tokens.index(ent.text) + 1) and (tokens.index(ent.text) > 0):
                words.append([tokens[tokens.index(ent.text) - 1] + ' ' + tokens[tokens.index(ent.text)] + ' ' + tokens[tokens.index(ent.text) + 1], ent.type])
                #print(ent.text, ent.type)
                #print(tokens[tokens.index(ent.text) - 1] + ' ' + tokens[tokens.index(ent.text)] + ' ' + tokens[tokens.index(ent.text) + 1])  
    words_for_model.extend(words)

In [63]:
df_threewords = pd.DataFrame(words_for_model, columns=['words', 'tag'])
df_threewords['tag'].value_counts()

PER         3526
GEOPOLIT    3264
ORG         2900
LOC         1439
MEDIA        306
Name: tag, dtype: int64

In [64]:
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(df_threewords['words'], df_threewords['tag'])

# labelEncode целевую переменную
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)

train_data = tf.data.Dataset.from_tensor_slices((train_x, train_y))
valid_data = tf.data.Dataset.from_tensor_slices((valid_x, valid_y))

train_data = train_data.batch(16)
valid_data = valid_data.batch(16)

AUTOTUNE = tf.data.AUTOTUNE

train_data = train_data.cache().prefetch(buffer_size=AUTOTUNE)
valid_data = valid_data.cache().prefetch(buffer_size=AUTOTUNE)

так как у нас получилось меньше классов немного поправим нашу NN

In [65]:
embedding_dim = 64

class model3NER(tf.keras.Model):
    def __init__(self):
        super(model3NER, self).__init__()
        self.emb = Embedding(vocab_size, embedding_dim)
        self.gPool = GlobalMaxPooling1D()
        self.fc1 = Dense(300, activation='relu')
        self.fc2 = Dense(50, activation='relu')
        self.fc3 = Dense(5, activation='softmax')

    def call(self, x):
        x = vectorize_layer(x)
        x = self.emb(x)
        pool_x = self.gPool(x)
        
        fc_x = self.fc1(pool_x)
        fc_x = self.fc2(fc_x)
        
        concat_x = tf.concat([pool_x, fc_x], axis=1)
        prob = self.fc3(concat_x)
        return prob

In [66]:
mmodel3 = model3NER()
mmodel3.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

In [67]:
mmodel3.fit(train_data, validation_data=valid_data, epochs=3)

Epoch 1/3
536/536 [==============================] - 15s 27ms/step - loss: 0.8426 - accuracy: 0.6947 - val_loss: 0.3747 - val_accuracy: 0.8783
Epoch 2/3
536/536 [==============================] - 14s 26ms/step - loss: 0.1760 - accuracy: 0.9451 - val_loss: 0.2936 - val_accuracy: 0.9143
Epoch 3/3
536/536 [==============================] - 14s 26ms/step - loss: 0.0625 - accuracy: 0.9824 - val_loss: 0.3011 - val_accuracy: 0.9154


In [68]:
# prediction on test data
y_pred = mmodel3.predict(valid_x)
y_pred_lbls = np.argmax(y_pred, -1)

In [69]:
# metrics for prediction
precision = sklearn.metrics.precision_score(valid_y, y_pred_lbls, average = 'weighted')
recall = sklearn.metrics.recall_score(valid_y, y_pred_lbls, average = 'weighted')
f1_score = sklearn.metrics.f1_score(valid_y, y_pred_lbls, average = 'weighted')
precision, recall, f1_score

(0.9158244468291882, 0.9153550192374956, 0.915175865577021)

#### Выводы по работе с собственной моделю определения NER
Создание своей модели определения NER не такая уж и сложная задача. Как всегда главное это нормально размеченный датасет, а все остальное уже второстепенно.
При работе различных вариаций моделей можно выделить следующее:
1. У модели с одним токеном F1 метрика выше чем у модели с 3 токенами
2. Вероятнее всего такой результат эксперимента связан с тем, что в "токен" из трех слов попадало много "мусора"
3. Стоить отметить что разница не существенная, и на реальной задаче мы бы этого не скорее всего не увидили бы
